<h3>Lemmatization and full text search (FTS)</h3>

Konrad Przewłoka


For this task a python Elastisearch library was used due to a rather simple api and general ease of use

<h4>Necessary imports</h4>

In [1]:
!pip install elasticsearch
import os
from elasticsearch import Elasticsearch

<h4>Creating elasticsearch client</h4>

In [2]:
#please kindly ignore those credentials, this ElasticSearch is no longer operational as of ending of this laboratory
client = Elasticsearch("https://localhost:9200",
                    #verify_certs=False,
                       ca_certs='../../elasticsearch-8.4.3//config/certs/http_ca.crt',
                       basic_auth=("elastic", 'zu02AJZclkOQ692G1dtr'))
client.info()

ObjectApiResponse({'name': 'KONRAD', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'oj3xJKcETvqZEUMdUAACiA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

<h4>Creating index and defining analyzer</h4>

In [11]:
 client.indices.create(
        index="ustawy_index",
       body={
            "settings":{  
                "analysis":{  
                    "analyzer":{  
                        "default":{  
                        "type":"custom",
                        "tokenizer":"standard",
                        "filter":[  
                            "synonyms",
                            "morfologik_stem",
                            "lowercase",
                        ]
                        }
                    },
                    "filter":{
                        "synonyms":{  
                        "type":"synonym",
                        "synonyms":[  
                            "kpk => kodeks postępowania karnego",
                            "kpc => kodeks postępowania cywilnego", 
                            "kk => kodeks karny",
                            "kc => kodeks cywilny"
                        ]}}}},})

C:\Users\KPR\AppData\Local\Temp\ipykernel_28536\939522864.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ustawy_index'})

<h4>Load data</h4>

In [69]:
files = os.listdir("../ustawy") 
for file in files:
    with open("../ustawy" + '/' + file, 'r', encoding='utf8') as f:
        tmp = f.read()
        client.create(index="ustawy_index", id=file, body={"data": tmp})


C:\Users\KPR\AppData\Local\Temp\ipykernel_1632\801060788.py:5: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.create(index="ustawy_index", id=file, body={"data": tmp})


<h4>Count number of files containing word ustawa</h4>

In [4]:
ustawy_result = client.search(
    index="ustawy_index",
    body={
        "query": {
            "match": {
                "data": {
                    "query": "ustawa"
                }}}})
ustawy_result["hits"]["total"]["value"]

C:\Users\KPR\AppData\Local\Temp\ipykernel_28536\140135700.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  ustawy_result = client.search(


1178

<h4>Counting occurences of word ustawa (all inflection forms)</h4>

In [74]:
occurence_result=client.termvectors(index="ustawy_index",
                   term_statistics=True,
                   field_statistics=True,
                   fields=["data"],
                   id="1993_599.txt")
occurence_result["term_vectors"]["data"]["terms"]["ustawa"]["ttf"]

24934

<h4>Count number of files containing phrase "kodeks postępowania cywilnego"</h4>

In [75]:
synonym_result=client.search(
    index="ustawy_index",
    body={
        "query": {
            "match_phrase": {
                "data": {
                    "query": "kodeks postępowania cywilnego"
                }}}})
synonym_result["hits"]["total"]["value"]

C:\Users\KPR\AppData\Local\Temp\ipykernel_1632\3244932394.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  synonym_result=client.search(


99

<h4>Count number of files containing phrase "wchodzi w życie"</h4>

In [78]:
phrase_result = client.search(
    index="ustawy_index",
    body={
        "query": {
            "match_phrase": {
                "data": {
                    "query": "wchodzi w życie",
                    "slop": 2
                }}}})
phrase_result["hits"]['total']['value']

C:\Users\KPR\AppData\Local\Temp\ipykernel_1632\3852270603.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  phrase_result = client.search(


1174

<h4>Top 10 documents for "konstytucja"</h4>

In [7]:
konstytucja_result=client.search(
    index="ustawy_index",
    body={
        "query": {
            "match": {
                "data": {
                    "query": "konstytucja"
                }}},
        "size": 10})

for i in range(10):
    print(konstytucja_result["hits"]["hits"][i]['_id'])

1997_629.txt
2000_443.txt
1997_604.txt
1996_350.txt
1997_642.txt
2001_23.txt
1996_199.txt
1999_688.txt
1997_681.txt
2001_1082.txt


C:\Users\KPR\AppData\Local\Temp\ipykernel_28536\865889251.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  konstytucja_result=client.search(


<h4>Top 3 highlights</h4>

In [9]:
konstytucja_result=client.search(
    index="ustawy_index",
    body={
        "query": {
            "match": {
                "data": {
                    "query": "konstytucja"
                }}},
        "size":10,
         "highlight": {
            "fields": {
                "data": {}
            },            
            "number_of_fragments": 3,
            "boundary_scanner": "sentence",

        }})
for i in range(10):
    for sentence  in konstytucja_result['hits']['hits'][i]['highlight']['data']:
        print(sentence)
        print('\n')

C:\Users\KPR\AppData\Local\Temp\ipykernel_28536\625406752.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  konstytucja_result=client.search(


o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej


W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em>


Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie


umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em>


międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>


co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em>


Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii 
   politycznej z <em>Konstytucją</em>


Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności z <em>Konstytucją</em>
   celów lub działalności


Ciężar udowodnienia niezgodności z <em>Konstytucją</em> spoc

In [10]:
client.indices.delete(index='ustawy_index')

ObjectApiResponse({'acknowledged': True})